<a href="https://colab.research.google.com/github/Red4mption/jets/blob/main/Hengkai_Jiang_19013119_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN for Jets

Hengkai Jiang 19013119


In [ ]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os

## Load the data

### original data file (run the code only if you don't have the data)

I have no idea what pt300, pt600, 0_600, 0_1500 mean, but all four files have the same dictionary and shape, so I just use all of them together.

In [ ]:
# uncomment the code if you need to run it
# I comment them so I don't run them everytime I restart my notebook

################################################################

# download and unzip the data if you don't have it

urllib.request.urlretrieve('http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/jetimage/20190920_partial_10k.zip', "20190920_partial_10k.zip")
with zipfile.ZipFile("20190920_partial_10k.zip","r") as zip_ref:
   zip_ref.extractall()

###############################################################

# load the data locally, change the path according to yours
pt300 = np.load("./20190920_partial_10k/20190920_pt300.0_600.0_40bins_10k.npz")
pt600 = np.load("./20190920_partial_10k/20190920_pt600.0_1500.0_40bins_10k.npz")
pt1500 = np.load("./20190920_partial_10k/20190920_pt1500.0_2500.0_40bins_10k.npz")
pt2500 = np.load("./20190920_partial_10k/20190920_pt2500.0_5000.0_40bins_10k.npz")


The task is to build a CNN using the images to identify W bosons jets and background jets.

The data file contains a lot more data than just images of jets above.

So I use only the jet images of W bosons and background to build a new file, reducing the amount of data loaded each time.

In [ ]:
# uncomment the code if you need to run it
# I comment them so I don't run them everytime I restart my notebook

# download and load the datafile by running code blocks above.

#concatenate all w jets data 
w_jets_data = np.concatenate(
   (pt300['W_jet_images'],pt600['W_jet_images'],pt1500['W_jet_images'],pt2500['W_jet_images']),axis = 0)

#concatenate all background jets data
background_jets_data =  np.concatenate(
   (pt300['QCD_jet_images'],pt600['QCD_jet_images'],pt1500['QCD_jet_images'],pt2500['QCD_jet_images']),axis = 0)

#w jets label
w_labels = np.vstack((
    np.ones(len(w_jets_data)),np.zeros(len(w_jets_data)))).T

#background jets label
background_labels = np.vstack((
    np.zeros(len(background_jets_data)),np.ones(len(background_jets_data)))).T 

jet_images = np.concatenate((w_jets_data,background_jets_data),axis = 0)
jet_labels = np.concatenate((w_labels,background_labels),axis = 0)

#shuffle the data without changing correspondence between two arrays
indices = np.arange(jet_images.shape[0])
np.random.shuffle(indices)
jet_images = jet_images[indices]
jet_labels = jet_labels[indices]

#separate the data into training set, test set and validation set
np.savez_compressed(
    "training_set.npz",images = jet_images[:int(0.7*len(jet_images))], 
    labels = jet_labels[:int(0.7*len(jet_images))])
np.savez_compressed(
    "test_set.npz",images = jet_images[int(0.7*len(jet_images)):int(0.85*len(jet_images))], 
    labels = jet_labels[int(0.7*len(jet_images)):int(0.85*len(jet_images))])
np.savez_compressed(
    "validation_set.npz",images = jet_images[int(0.85*len(jet_images)):], 
    labels = jet_labels[int(0.85*len(jet_images)):])

### New data files (run codes above to generate them)

I had considered upload the data files online but I don't know if it is appropriate to do so. So, I include the code for generating in case you would like to run it yourself.

In [ ]:
# change the path accordingly, generate the file using code blocks above
training_set = np.load("training_set.npz")
training_data = training_set['images'].reshape(training_set['images'].shape[0],40,40,1).astype('float32')
training_label = training_set['labels']
test_set = np.load("test_set.npz")
test_data = test_set['images'].reshape(test_set['images'].shape[0],40,40,1).astype('float32')
test_label = test_set['labels']
validation_set = np.load("validation_set.npz")
validation_data = validation_set['images'].reshape(validation_set['images'].shape[0],40,40,1).astype('float32')
validation_label = validation_set['labels']


## The CNN

In [23]:
############################################

# the architecture of the model

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(30,(3,3),input_shape = (40,40,1)))
model.add(tf.keras.layers.PReLU())
model.add(tf.keras.layers.Conv2D(30,(3,3)))
model.add(tf.keras.layers.PReLU())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200, activation="sigmoid"))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(2))

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              optimizer = 'adam',metrics=['accuracy'])

############################################

# load the model 
# model = tf.keras.models.load_model()

In [24]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 38, 38, 30)        300       
                                                                 
 p_re_lu_6 (PReLU)           (None, 38, 38, 30)        43320     
                                                                 
 conv2d_7 (Conv2D)           (None, 36, 36, 30)        8130      
                                                                 
 p_re_lu_7 (PReLU)           (None, 36, 36, 30)        38880     
                                                                 
 dropout_7 (Dropout)         (None, 36, 36, 30)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 30)       0         
 2D)                                                             
                                                      

In [27]:
history=model.fit(training_data, training_label,batch_size=100, epochs=20, validation_data=(validation_data,validation_label))


Epoch 1/20
560/560 [==============================] - 17s 30ms/step - loss: 0.5099 - accuracy: 0.7511 - val_loss: 0.5289 - val_accuracy: 0.7351
Epoch 2/20
560/560 [==============================] - 17s 30ms/step - loss: 0.5107 - accuracy: 0.7527 - val_loss: 0.5291 - val_accuracy: 0.7395
Epoch 3/20
560/560 [==============================] - 17s 30ms/step - loss: 0.5112 - accuracy: 0.7529 - val_loss: 0.5322 - val_accuracy: 0.7368
Epoch 4/20
560/560 [==============================] - 17s 30ms/step - loss: 0.5132 - accuracy: 0.7512 - val_loss: 0.5376 - val_accuracy: 0.7281
Epoch 5/20
560/560 [==============================] - 17s 30ms/step - loss: 0.5150 - accuracy: 0.7479 - val_loss: 0.5348 - val_accuracy: 0.7337
Epoch 6/20
560/560 [==============================] - 17s 30ms/step - loss: 0.5123 - accuracy: 0.7511 - val_loss: 0.5282 - val_accuracy: 0.7364
Epoch 7/20
560/560 [==============================] - 17s 30ms/step - loss: 0.5121 - accuracy: 0.7509 - val_loss: 0.5397 - val_accuracy:

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.6408880239999917
GPU (s):
0.05386709199999018
GPU speedup over CPU: 67x


In [ ]:
arr = np.vstack((np.zeros(3),np.ones(3))).T
arr2 = np.vstack((np.ones(3),np.zeros(3))).T
print(np.concatenate((arr,arr2),axis = 0))

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
